# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!jupyter nbextension enable --py --sys-prefix gmaps

# Import API key
from api_keys import g_key

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather_df = pd.read_csv("../output_data/cities.csv")
city_weather_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Mataura,-46.1927,168.8643,59.49,65,14,2.73,NZ,1676669088
1,1,Airai,-8.9266,125.4092,59.52,87,100,1.05,TL,1676668925
2,2,Dikson,73.5069,80.5464,18.00,95,100,19.82,RU,1676669089
3,3,Upernavik,72.7868,-56.1549,-20.29,98,32,10.09,GL,1676669089
4,4,Umluj,25.0213,37.2685,69.37,45,100,17.60,SA,1676668926


In [3]:
city_weather_df["Lat"].count()

139

In [4]:
city_weather_df["Lng"].count()

139

In [5]:
city_weather_df["Humidity"].count()

139

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
#For API Key in gmaps
gmaps.configure(api_key=g_key) 

In [7]:
coordinates = city_weather_df[['Lat','Lng']]
humidity = city_weather_df['Humidity'].astype(float)

In [10]:
#create the heatmap
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(coordinates,weights=humidity,dissipating=False, max_intensity=100,                                point_radius = 1)
heatmap_layer.max_intensity = 70
heatmap_layer.point_radius = 3

fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
#weather conditions:
#1.max temp < 80 but > 70
#2.wind speed < 10 mph
#3.zero cloudiness
#drop any rows that do not satisfy all three conditions

perfect_weather = city_weather_df.loc[(city_weather_df['Max Temp'] < 80)& 
                                (city_weather_df['Max Temp'] > 70)&
                                (city_weather_df['Wind Speed'] < 10)&
                                (city_weather_df['Cloudiness'] == 0)]

perfect_weather.count()

Unnamed: 0    1
City          1
Lat           1
Lng           1
Max Temp      1
Humidity      1
Cloudiness    1
Wind Speed    1
Country       1
Date          1
dtype: int64

In [16]:
#drop rows with null values
perfect_temperature = perfect_weather.dropna(how='any')
perfect_temperature.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,Manthani,18.65,79.6667,70.75,32,0,3.18,IN,1676668927


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
#add a hotel name column 
hotels = perfect_temperature
hotels['Hotel Name'] = None
hotels

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
9,9,Manthani,18.65,79.6667,70.75,32,0,3.18,IN,1676668927,None


In [19]:
# link : https://maps.googleapis.com/maps/api/place/nearbysearch/json?parameters

params = {
    "radius": 10000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in perf_temp.iterrows():
    #getting the lat and lng value from df
    lat= row['Lat']
    lng= row['Lng']
    params['location'] = f'{lat},{lng}'
    #base url for the request
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    #making requests to get response
    hotel_info = requests.get(base, params = params).json()
    #try and except
    try:
        perf_temp.loc[index,'Hotel Name'] = hotel_name['results'][0]['name']
    except:
        perf_temp.loc[index,'Hotel Name'] = 'No Hotel'

/var/folders/bp/kkf4c1nj557c5frmyn420c7c0000gn/T/ipykernel_51630/1436498036.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perf_temp.loc[index,'Hotel Name'] = 'No Hotel'


In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [23]:
# Add marker layer ontop of heat map
markers_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))